In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import SpectralClustering, KMeans

with open('embeddings.pkl', 'rb') as file:
    data = pickle.load(file)

x = data["outputs"]
y = data["labels"]
embeddings = np.array(x)
y = np.array(y)

In [68]:
pca = PCA(n_components=135)
pca_data = pca.fit_transform(embeddings)

In [69]:
dct = {i:[] for i in range(100)}
for i in tqdm(range(100)):
    for j in range(y.shape[0]):
        if i == y[j]:
            dct[i].append(pca_data[j])

dct = {i:np.array(dct[i]) for i in range(100)}
dct = {i : np.mean(dct[i], axis=0) for i in range(100)}


100%|██████████| 100/100 [00:00<00:00, 140.82it/s]


In [70]:
X = np.array([dct[i] for i in range(100)])
clustering = KMeans(n_clusters=10, max_iter=1000).fit(X)

/home/farkhad.akimov/.conda/envs/701p/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [71]:
np.unique(clustering.labels_)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [72]:
clustering.labels_

array([0, 9, 4, 2, 2, 7, 9, 9, 8, 7, 7, 4, 6, 8, 9, 1, 7, 6, 9, 1, 7, 2,
       7, 6, 0, 7, 9, 2, 7, 1, 2, 1, 2, 5, 1, 4, 2, 6, 1, 6, 7, 8, 1, 1,
       9, 9, 4, 5, 8, 6, 2, 0, 5, 0, 3, 2, 5, 0, 8, 5, 6, 7, 3, 2, 2, 2,
       2, 2, 6, 6, 3, 6, 2, 2, 2, 2, 6, 0, 9, 9, 2, 8, 3, 3, 7, 8, 7, 7,
       1, 8, 8, 9, 3, 9, 7, 2, 5, 1, 4, 9], dtype=int32)

In [73]:
dct.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [75]:
new_dct = {i:[] for i in range(10)}
for i, v in enumerate(clustering.labels_):
    new_dct[v].append(list(dct.keys())[i])
print(new_dct)


{0: [0, 24, 51, 53, 57, 77], 1: [15, 19, 29, 31, 34, 38, 42, 43, 88, 97], 2: [3, 4, 21, 27, 30, 32, 36, 50, 55, 63, 64, 65, 66, 67, 72, 73, 74, 75, 80, 95], 3: [54, 62, 70, 82, 83, 92], 4: [2, 11, 35, 46, 98], 5: [33, 47, 52, 56, 59, 96], 6: [12, 17, 23, 37, 39, 49, 60, 68, 69, 71, 76], 7: [5, 9, 10, 16, 20, 22, 25, 28, 40, 61, 84, 86, 87, 94], 8: [8, 13, 41, 48, 58, 81, 85, 89, 90], 9: [1, 6, 7, 14, 18, 26, 44, 45, 78, 79, 91, 93, 99]}


In [76]:
def get_bin_centroid(X, lst):
    l = []
    for ind in lst:
        l.append(list(X[ind]))
    l = np.array(l)
    return np.mean(l, axis = 0)

centroids_dct = {i : get_bin_centroid(X, new_dct[i]) for i in range(10)}


In [81]:
import numpy as np

def balance_clusters(X, new_dct, centroids_dct):
   
    while any(len(classes) < 10 for classes in new_dct.values()):
        print(new_dct)
        print('-----------------------------\n\n\n')
        start, end, min_dist, ind = -1, -1, np.inf, -1
        for cluster, lst_classes in new_dct.items():
            if len(lst_classes) > 10:
                for i in range(10):
                    if i != cluster and len(new_dct[i]) < 10:
                        for clss in lst_classes:
                            d = np.linalg.norm(X[clss] - centroids_dct[i])
                            if d < min_dist:
                                start = cluster
                                end = i
                                ind = clss
                                break
        new_dct[start].remove(ind)
        new_dct[end].append(ind)
    return new_dct






In [83]:
d = balance_clusters(X, new_dct, centroids_dct)

In [85]:
d

{0: [0, 24, 51, 53, 57, 77, 36, 50, 55, 63],
 1: [15, 19, 29, 31, 34, 38, 42, 43, 88, 97],
 2: [64, 65, 66, 67, 72, 73, 74, 75, 80, 95],
 3: [54, 62, 70, 82, 83, 92, 21, 27, 30, 32],
 4: [2, 11, 35, 46, 98, 10, 16, 12, 3, 4],
 5: [33, 47, 52, 56, 59, 96, 6, 7, 5, 9],
 6: [17, 23, 37, 39, 49, 60, 68, 69, 71, 76],
 7: [20, 22, 25, 28, 40, 61, 84, 86, 87, 94],
 8: [8, 13, 41, 48, 58, 81, 85, 89, 90, 1],
 9: [14, 18, 26, 44, 45, 78, 79, 91, 93, 99]}

In [86]:
d_for_exp = {}
for k, v in d.items():
    for item in v:
        d_for_exp[item] = k
print(d_for_exp)

{0: 0, 24: 0, 51: 0, 53: 0, 57: 0, 77: 0, 36: 0, 50: 0, 55: 0, 63: 0, 15: 1, 19: 1, 29: 1, 31: 1, 34: 1, 38: 1, 42: 1, 43: 1, 88: 1, 97: 1, 64: 2, 65: 2, 66: 2, 67: 2, 72: 2, 73: 2, 74: 2, 75: 2, 80: 2, 95: 2, 54: 3, 62: 3, 70: 3, 82: 3, 83: 3, 92: 3, 21: 3, 27: 3, 30: 3, 32: 3, 2: 4, 11: 4, 35: 4, 46: 4, 98: 4, 10: 4, 16: 4, 12: 4, 3: 4, 4: 4, 33: 5, 47: 5, 52: 5, 56: 5, 59: 5, 96: 5, 6: 5, 7: 5, 5: 5, 9: 5, 17: 6, 23: 6, 37: 6, 39: 6, 49: 6, 60: 6, 68: 6, 69: 6, 71: 6, 76: 6, 20: 7, 22: 7, 25: 7, 28: 7, 40: 7, 61: 7, 84: 7, 86: 7, 87: 7, 94: 7, 8: 8, 13: 8, 41: 8, 48: 8, 58: 8, 81: 8, 85: 8, 89: 8, 90: 8, 1: 8, 14: 9, 18: 9, 26: 9, 44: 9, 45: 9, 78: 9, 79: 9, 91: 9, 93: 9, 99: 9}
